# Building a Custom Classifier in Keras with Transfer Learning

Steps shown below:

- **Organize the data**: Download labeled images of cats and dogs from Kaggle. Then divide the images into training and validation folders.
- **Set up the configuration**: Define a pipeline for reading data, including preprocessing the images (e.g. resizing) and batching multiple images together.
- **Load and augment the data**: In the absence of a ton of training images, make small changes (augmentation) like rotation, zooming, etc to increase variation in training data.
- **Define the model**: Take a pre-trained model, remove the last few layers, and append a new classifier layer. Freeze the weights of original layers (i.e. make them unmodifiable). Select an optimizer algorithm and a metric to track (like accuracy).
- **Train and test**: Start training for a few iterations. Save the model to eventually load inside any application for predictions.

## Organize the data

Before training, we need to store our [downloaded dataset](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/download/train.zip) in the right folder structure. Remember to make the `data` directory where we will be performing the refactoring. We’ll divide the images into two sets – training and validation. Our directory structure will look something like this: 

```
data
 |__train
 |    |__cat
 |    |__dog
 |__val
      |__cat
      |__dog
```

In Windows, the following lines of command can help achieve this directory structure:

In [1]:
import zipfile
import os
import random
import shutil

# Define the relative path to the train.zip file
zip_file_path = os.path.join('..', '..', 'Downloads', 'dogs-vs-cats-redux-kernels-edition', 'train.zip')

# Define the directory for extraction
data_dir = 'data'  # Main directory for extraction and work
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)  # Extract directly to 'data'

# Check that the files were extracted to data/train
train_files_dir = os.path.join(data_dir, 'train')
if not os.path.exists(train_files_dir):
    print("Error: The 'train' folder was not created.")
else:
    print("Files extracted to the 'train' folder.")

# Create necessary directories
os.makedirs(os.path.join(data_dir, 'train', 'cat'), exist_ok=True)
os.makedirs(os.path.join(data_dir, 'train', 'dog'), exist_ok=True)
os.makedirs(os.path.join(data_dir, 'val', 'cat'), exist_ok=True)
os.makedirs(os.path.join(data_dir, 'val', 'dog'), exist_ok=True)

print("Directories 'train' and 'val' successfully created.")

# Create a list of all files in the train directory (where the files were extracted)
all_files = os.listdir(train_files_dir)

# Filter files related to cats and dogs
cat_files = [f for f in all_files if 'cat' in f and f.endswith('.jpg')]
dog_files = [f for f in all_files if 'dog' in f and f.endswith('.jpg')]

# Shuffle the lists randomly
random.shuffle(cat_files)
random.shuffle(dog_files)

# Select 250 files for each category and move them to the train folders
for f in cat_files[:250]:
    shutil.move(os.path.join(train_files_dir, f), os.path.join(data_dir, 'train', 'cat', f))
for f in dog_files[:250]:
    shutil.move(os.path.join(train_files_dir, f), os.path.join(data_dir, 'train', 'dog', f))

# Use the remaining files for the val (validation) folders
for f in cat_files[250:500]:
    shutil.move(os.path.join(train_files_dir, f), os.path.join(data_dir, 'val', 'cat', f))
for f in dog_files[250:500]:
    shutil.move(os.path.join(train_files_dir, f), os.path.join(data_dir, 'val', 'dog', f))

print("Files successfully distributed into 'train' and 'val' folders.")

# Output absolute paths to the train and val directories
train_dir_path = os.path.abspath(os.path.join(data_dir, 'train'))
val_dir_path = os.path.abspath(os.path.join(data_dir, 'val'))

print(f"Path to the 'train' directory: {train_dir_path}")
print(f"Path to the 'val' directory: {val_dir_path}")


Файлы распакованы в папку 'train'.
Директории train и val успешно созданы.
Файлы успешно распределены по папкам train и val.
Путь к директории train: c:\Users\Валерия\Documents\GitHub\Practical-Deep-Learning-Book\code\chapter-3\data\train
Путь к директории val: c:\Users\Валерия\Documents\GitHub\Practical-Deep-Learning-Book\code\chapter-3\data\val


In [2]:
print(f"Images in train/cat: {len(os.listdir(os.path.join(data_dir, 'train', 'cat')))}")
print(f"Images in train/dog: {len(os.listdir(os.path.join(data_dir, 'train', 'dog')))}")
print(f"Images in val/cat: {len(os.listdir(os.path.join(data_dir, 'val', 'cat')))}")
print(f"Images in val/dog: {len(os.listdir(os.path.join(data_dir, 'val', 'dog')))}")

Images in train/cat: 250
Images in train/dog: 250
Images in val/cat: 250
Images in val/dog: 250


The 25,000 files inside the data folder are prefixed with ‘cat’ and ‘dog’. Now, move the files into their respective directories. To keep our initial experiment short, we’ll pick 250 random files per class and place them in training and validation folders. You can increase/decrease this number anytime, to experiment with a trade-off between accuracy and speed. 

Classification accuracy on previously unseen images (in the validation folder) is a good proxy for how the classifier would perform in the real world. Ideally, the more training images, the better the learning will be. And, the more validation images, the better our classifier would perform in the real-world.

## Set up the configuration

Let's start off with our Python program and begin with importing the necessary packages.

In [5]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.applications.mobilenet import MobileNet, preprocess_input
import math

Let's place all the configurations up-front. These can be modified in the future based on the dataset of your choice.

In [6]:
TRAIN_DATA_DIR = os.path.join(data_dir, 'train')
VALIDATION_DATA_DIR = os.path.join(data_dir, 'val')
TRAIN_SAMPLES = 500
VALIDATION_SAMPLES = 500
NUM_CLASSES = 2
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 64

## Load and augment the data

Colored images usually have 3 channels viz. red, green and blue, each with intensity value ranging from 0 to 255. To normalize it (i.e. bring the value between 0 to 1), we can rescale the image by dividing each pixel by 255. Or, we can use the default `preprocess_input` function in Keras which does the preprocessing for us.

In [7]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)
val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

Time to load the data from its directories and let the augmentation happen! 

A few key things to note:

- Training one image at a time can be pretty inefficient, so we can batch them into groups. 
- To introduce more randomness during the training process, we’ll keep shuffling the images in each batch.
- To bring reproducibility during multiple runs of the same program, we’ll give the random number generator a seed value.

In [8]:
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_DIR,
                                                    target_size=(IMG_WIDTH,
                                                                 IMG_HEIGHT),
                                                    batch_size=BATCH_SIZE,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')
validation_generator = val_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


Now that the data is taken care of, we come to the most crucial component of our training process - the model. We will reuse a CNN previously trained on the ImageNet dataset, remove the ImageNet specific classifier in the last few layers, and replace it with our own classifier suited to our problem. For transfer learning, we’ll ‘freeze’ the weights of the original model, i.e. set those layers as unmodifiable, so only the layers of the new classifier (that we add) can be modified. To keep things fast, we’ll choose the MobileNet model.

## Define the model

In [9]:
def model_maker():
    base_model = MobileNet(include_top=False,
                           input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        layer.trainable = False
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

## Train and test

With both the data and model ready, all we have left to do is train the model. This is also known as fitting the model to the data. For training any model, we need to pick a loss function, an optimizer, initial learning rate and a metric.

In [10]:
model = model_maker()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['acc'])
model.fit_generator(
    train_generator,
    steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / BATCH_SIZE),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=math.ceil(float(VALIDATION_SAMPLES) / BATCH_SIZE))

C:\Users\Валерия\AppData\Local\Temp\ipykernel_12208\2507997108.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/10


8/8 [==============================] - 38s 4s/step - loss: 0.9497 - acc: 0.6140 - val_loss: 0.1834 - val_acc: 0.9420
Epoch 2/10
8/8 [==============================] - 14s 2s/step - loss: 0.3160 - acc: 0.8720 - val_loss: 0.1116 - val_acc: 0.9640
Epoch 3/10
8/8 [==============================] - 14s 2s/step - loss: 0.2070 - acc: 0.9300 - val_loss: 0.0872 - val_acc: 0.9640
Epoch 4/10
8/8 [==============================] - 14s 2s/step - loss: 0.1605 - acc: 0.9380 - val_loss: 0.0749 - val_acc: 0.9720
Epoch 5/10
8/8 [==============================] - 13s 2s/step - loss: 0.1261 - acc: 0.9540 - val_loss: 0.0677 - val_acc: 0.9740
Epoch 6/10
8/8 [==============================] - 13s 2s/step - loss: 0.1308 - acc: 0.9560 - val_loss: 0.0648 - val_acc: 0.9740
Epoch 7/10
8/8 [==============================] - 14s 2s/step - loss: 0.0978 - acc: 0.9620 - val_loss: 0.0625 - val_acc: 0.9720
Epoch 8/10
8/8 [==============================] - 13s 2s/step - loss: 0.1238 - acc: 0.9620 - val_loss:

By the 10th step, we observe about 97% validation accuracy.


In [11]:
model.save('model.h5')

c:\Users\Валерия\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Model Prediction

We can now load this model anytime and classify an image. The Keras function `load_model`, as the name suggests loads the model. 

In [12]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
model = load_model('model.h5')

# Check the current working directory
current_dir = os.getcwd()
print("Current working directory:", current_dir)

# Check if the file model.h5 exists
file_exists = os.path.isfile('model.h5')
print("File 'model.h5' exists:", file_exists)

# Listing files in the current directory
os.listdir(current_dir)

Now let’s try loading our original sample images and see what results we get.

In [22]:
img_path = os.path.join('data', 'val', 'dog', 'dog.3714.jpg')
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img_array, axis=0)
preprocessed_img = expanded_img_array / 255.  # Preprocess the image
prediction = model.predict(preprocessed_img)
print(prediction)
print(validation_generator.class_indices)

1/1 [==============================] - 1s 638ms/step
[[7.4766518e-04 9.9925226e-01]]
{'cat': 0, 'dog': 1}


## Time to analyze the Model

Note: I import packages one more time to avoid some errors on Windows. 

In [ ]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


IMG_WIDTH, IMG_HEIGHT = 224, 224
VALIDATION_DATA_DIR = os.path.join('data', 'val')
VALIDATION_BATCH_SIZE = 64

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=VALIDATION_BATCH_SIZE,
    shuffle=False,
    class_mode='categorical')

from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

with CustomObjectScope(
    {'GlorotUniform': glorot_uniform()}):
    model_path = os.path.join('data', 'model.h5')
    
# Experiments start here
ground_truth = validation_generator.classes

predictions = model.predict(validation_generator)

# Create a dictionary of image index: true prediction
prediction_table = {}
for index, val in enumerate(predictions):
    index_of_highest_probability = np.argmax(val)
    value_of_highest_probability = val[index_of_highest_probability]
    prediction_table[index] = [value_of_highest_probability,
                               index_of_highest_probability,
                               ground_truth[index]]

assert len(predictions) == len(ground_truth) == len(prediction_table)

def display(sorted_indices, message):
    similar_image_paths = []
    distances = []
    for name, value in sorted_indices:
        [probability, predicted_index, gt] = value
        similar_image_paths.append(VALIDATION_DATA_DIR + fnames[name])
        distances.append(probability)
    plt.subplots(similar_image_paths, distances, message)

# Predictions for the 'dog' class with the highest probability values
indices = get_images_with_sorted_probabilities(prediction_table,
                                               get_highest_probability=True,
                                               label=1, number_of_items=10,
                                               only_false_predictions=False)
message = 'Images with the highest probability of containing dogs'
display(indices[:10], message)


Next checkpoint is to identify our best and worst predictions.

In [ ]:
# Predictions about the 'dog' class with the highest probability values
indices = get_images_with_sorted_probabilities(prediction_table,
get_highest_probability=True, label=1, number_of_items=10,
only_false_predictions=False)
message = 'Images with the highest probability of containing dogs'
display(indices[:10], message)

In [ ]:
# Predictions about the 'dog' class with low probability values
low_prob_dog_indices = get_images_with_sorted_probabilities(prediction_table,
                                                            get_highest_probability=False, 
                                                            label=1,
                                                            number_of_items=10, 
                                                            only_false_predictions=False)

message = 'Images with the lowest probability of containing dogs'
display(low_prob_dog_indices, message)

In [ ]:
# Erroneous predictions about the 'dog' class
wrong_dog_indices = get_images_with_sorted_probabilities(prediction_table,
                                                         get_highest_probability=True, 
                                                         label=1,
                                                         number_of_items=10, 
                                                         only_false_predictions=True)

message = 'Wrong predictions for images of dogs'
display(wrong_dog_indices, message)

In [ ]:
# Predictions about the 'cat' class with the highest probability values
high_prob_cat_indices = get_images_with_sorted_probabilities(prediction_table,
                                                             get_highest_probability=True, 
                                                             label=0,
                                                             number_of_items=10, 
                                                             only_false_predictions=False)

message = 'Images with the highest probability of containing cats'
display(high_prob_cat_indices, message)

In [ ]:
# Predictions about the 'cat' class with low probability values
low_prob_cat_indices = get_images_with_sorted_probabilities(prediction_table,
                                                            get_highest_probability=False, 
                                                            label=0,
                                                            number_of_items=10, 
                                                            only_false_predictions=False)

message = 'Images with the lowest probability of containing cats'
display(low_prob_cat_indices, message)

In [ ]:
# Erroneous predictions about the 'cat' class
wrong_cat_indices = get_images_with_sorted_probabilities(prediction_table,
                                                         get_highest_probability=True, 
                                                         label=0,
                                                         number_of_items=10, 
                                                         only_false_predictions=True)

message = 'Wrong predictions for images of cats'
display(wrong_cat_indices, message)